<a href="https://colab.research.google.com/github/cincysam6/Field_Control_Model/blob/main/interactive_field_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clean out any conflicting versions
%pip -q uninstall -y ipywidgets widgetsnbextension jupyterlab_widgets || true

# Install a known-good combo for Colab classic
%pip -q install ipywidgets==7.7.1




In [ ]:
# 1. Clone your repo
#!git clone https://github.com/cincysam6/Field_Control_Model.git

In [7]:


# 2. Add it to Python path
import sys
sys.path.append('/content/Field_Control_Model')

# 3. Import your helper function
from src.helpers import compute_distances_by_frame, pick_panel_kwargs,update_kwargs
from src.diagnostic_plots import plot_dir_orientation_small_multiples,diagnostic_plot,plot_field_control_small_multiples,diagnostic_multiples
from src.field_control_model import PlayerInfluenceModel
from src.compute_player_density import compute_player_densities_dataframe
from src.plot_player_density import plot_player_densities_from_dataframe, plot_team_densities_small_multiples
from src.animate_plays import animate_pitch_control_with_players, animate_pitch_control_with_players_fast
from src.compute_player_density import compute_player_densities_team_control
from src.plot_player_density import visualize_team_control
from src.presets import default_kwargs, triangular_kwargs

In [8]:
from google.colab import output
output.enable_custom_widget_manager()

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import FancyArrow
from matplotlib.lines import Line2D
import ipywidgets as w
from ipywidgets import Layout, VBox, HBox, Accordion, Tab, Output, AppLayout
from IPython.display import display, clear_output


In [10]:
# ---------- Presets ----------
PRESETS = {
    "Default1": dict(
        gaussian_scale_factor=0.7,
        alpha_gamma=11.0,
        beta_min=1.0,
        beta_max=20.0,
        gamma_midpoint=15.0,
        gamma_scale_factor=0.8,
        max_forward_distance=20.0,
        forward_decay_factor=1.0,
        angle_limit_min=15.0,
        angle_limit_max=45.0,
        angle_decay_factor=2.0,
        w_gaussian_min=0.2,
        w_gaussian_max=1.0,
        gaussian_midpoint=4.0,
        gaussian_steepness=2.0,
        low_speed_gaussian_cutoff=2.0,  # <-- add this so apply_preset never KeyErrors
    ),
    "Default2": dict(
        gaussian_scale_factor=0.7,
        alpha_gamma=11.0,
        beta_min=2.0, beta_max=15.0,
        gamma_midpoint=15.0,
        gamma_scale_factor=0.8,
        max_forward_distance=18.0,
        forward_decay_factor=1.0,
        angle_limit_min=20.0, angle_limit_max=50.0, angle_decay_factor=2.5,
        w_gaussian_min=0.3, w_gaussian_max=1.0,
        gaussian_midpoint=4.5, gaussian_steepness=2.0,
        low_speed_gaussian_cutoff=2.25,
    ),
}

# ---------- Controls ----------
# Scene
speed      = w.FloatSlider(5.0,  min=0.0, max=11.5, step=0.1, description="Speed")
dir_deg    = w.FloatSlider(0.0,  min=0.0, max=360.0, step=1.0, description="Direction°")
show_o     = w.Checkbox(False, description="Show Orientation")
o_deg      = w.FloatSlider(0.0,  min=0.0, max=360.0, step=1.0, description="Orient°")
x          = w.FloatSlider(60.0, min=0.0, max=120.0, step=0.5, description="X")
y          = w.FloatSlider(26.65,min=0.0, max=53.3, step=0.5, description="Y")
name       = w.Text(value="Adam Thielen", description="Name")

# Gaussian
gaussian_scale_factor     = w.FloatSlider(0.68, min=0.4, max=1.2, step=0.02, description="Scale")
low_speed_gaussian_cutoff = w.FloatSlider(2.0,  min=0.0, max=5.0, step=0.1,  description="Low-spd cutoff")

# Gamma
alpha_gamma        = w.FloatSlider(6.0,  min=2.0,  max=12.0, step=0.5, description="alpha")
beta_min           = w.FloatSlider(2.0,  min=0.5,  max=8.0,  step=0.5, description="beta_min")
beta_max           = w.FloatSlider(18.0, min=8.0,  max=30.0, step=1.0, description="beta_max")
gamma_midpoint     = w.FloatSlider(13.0, min=6.0,  max=20.0, step=0.5, description="midpoint")
gamma_scale_factor = w.FloatSlider(0.9,  min=0.6,  max=1.4, step=0.02, description="scale_fac")
max_forward_distance = w.FloatSlider(20.0, min=10.0, max=30.0, step=1.0, description="cap")
forward_decay_factor = w.FloatSlider(1.1, min=0.6, max=2.0, step=0.05, description="tail_decay")

# Cone
angle_limit_min    = w.FloatSlider(15.0, min=8.0,  max=30.0, step=1.0, description="min° (fast)")
angle_limit_max    = w.FloatSlider(55.0, min=35.0, max=80.0, step=1.0, description="max° (slow)")
angle_decay_factor = w.FloatSlider(2.2,  min=1.0,  max=4.0,  step=0.1, description="decay")

# Weights
w_gaussian_min     = w.FloatSlider(0.2, min=0.0, max=0.5, step=0.02, description="wG min")
w_gaussian_max     = w.FloatSlider(1.0, min=0.5, max=1.0, step=0.02, description="wG max")
gaussian_midpoint  = w.FloatSlider(4.0, min=2.0, max=6.0, step=0.1, description="midpoint")
gaussian_steepness = w.FloatSlider(1.8, min=0.5, max=3.0, step=0.1, description="steepness")

# Style
levels     = w.IntSlider(24, min=10, max=60, step=1, description="Levels")
cmap       = w.Dropdown(options=["OrRd","Reds","magma","inferno"], value="OrRd", description="Colormap")
fill_alpha = w.FloatSlider(0.22, min=0.05, max=0.5, step=0.01, description="Fill α")
line_alpha = w.FloatSlider(0.50, min=0.1,  max=0.9, step=0.01, description="Line α")
line_lw    = w.FloatSlider(0.6,  min=0.2,  max=2.0, step=0.1,  description="Line lw")

# Preset & buttons (ONE print button only)
preset_dd  = w.Dropdown(options=list(PRESETS.keys()), value="Default1", description="Preset")
apply_btn  = w.Button(description="Apply Preset", button_style="info")
reset_btn  = w.Button(description="Reset Sliders", button_style="warning")
print_btn  = w.Button(description="Print Params", button_style="success")  # << single definition

# A dedicated output panel for printed params
params_out = Output(layout=Layout(border="1px solid #ddd", max_height="200px", overflow="auto"))

# Play (native ipywidgets control) — sweeps speed 0→11.5 by 0.5
play = w.Play(interval=150, value=0, min=0, max=int((11.5 - 0.0)/0.5), step=1, description="Press play")
play_box = w.HBox([w.Label("Play speed sweep"), play])

# ---------- Left panel (no Tabs/Accordion) ----------
def section(title, *kids):
    return VBox([w.HTML(f"<b style='font-family:system-ui'>{title}</b>"), VBox(list(kids))])

left_panel = VBox(
    [
        HBox([preset_dd, apply_btn, reset_btn, print_btn]),
        section("Scene", name, speed, dir_deg, show_o, o_deg, x, y, play_box),
        section("Gaussian", gaussian_scale_factor, low_speed_gaussian_cutoff),
        section("Gamma", alpha_gamma, beta_min, beta_max, gamma_midpoint, gamma_scale_factor,
                max_forward_distance, forward_decay_factor),
        section("Cone", angle_limit_min, angle_limit_max, angle_decay_factor),
        section("Weights", w_gaussian_min, w_gaussian_max, gaussian_midpoint, gaussian_steepness),
        section("Style", levels, cmap, fill_alpha, line_alpha, line_lw),
        w.HTML("<b style='font-family:system-ui'>Params output</b>"),
        params_out,
    ],
    layout=Layout(width="360px", height="1440px", overflow_y="auto", padding="6px 6px 6px 0")
)

# ---------- Plot output on the right ----------
out = Output(layout=Layout(border="1px solid #eee", height="720px"))

# ---------- helpers ----------
def _add_arrow(ax, start, theta_deg, length=2.2, body=0.08, head_w=0.35, head_l=0.70, color="#2F6CFF", z=6):
    x0, y0 = start
    th = np.deg2rad(90 - theta_deg)
    dx, dy = length*np.cos(th), length*np.sin(th)
    ax.add_patch(FancyArrow(
        x0, y0, dx, dy,
        width=body, head_width=head_w, head_length=head_l,
        length_includes_head=True, color=color, alpha=0.9, linewidth=0.6, zorder=z
    ))

def _draw_field_window(ax, extent):
    xmin, xmax, ymin, ymax = extent
    ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)
    ax.set_aspect('equal', adjustable='box')
    ax.set_facecolor("#FAFBFD")
    for xv in np.arange(np.floor(xmin), np.ceil(xmax)+1, 5):
        ax.axvline(xv, color="#D7DEE8", lw=0.6, alpha=0.6, zorder=0)
    for yv in np.arange(np.floor(ymin), np.ceil(ymax)+1, 5):
        ax.axhline(yv, color="#EFF3F8", lw=0.6, alpha=0.6, zorder=0)
    for sp in ax.spines.values():
        sp.set_visible(False)
    ax.tick_params(colors="#8A94A6", labelsize=9)
    ax.set_xlabel("X (yards)", color="#444B59")
    ax.set_ylabel("Y (yards)", color="#444B59")

def _current_model_kwargs():
    return dict(
        gaussian_scale_factor=gaussian_scale_factor.value,
        alpha_gamma=alpha_gamma.value,
        beta_min=beta_min.value, beta_max=beta_max.value,
        gamma_midpoint=gamma_midpoint.value,
        gamma_scale_factor=gamma_scale_factor.value,
        max_forward_distance=max_forward_distance.value,
        forward_decay_factor=forward_decay_factor.value,
        angle_limit_min=angle_limit_min.value,
        angle_limit_max=angle_limit_max.value,
        angle_decay_factor=angle_decay_factor.value,
        w_gaussian_min=w_gaussian_min.value,
        w_gaussian_max=w_gaussian_max.value,
        gaussian_midpoint=gaussian_midpoint.value,
        gaussian_steepness=gaussian_steepness.value,
        low_speed_gaussian_cutoff=low_speed_gaussian_cutoff.value,
    )

# Evaluate model on a local window: 10L,10R,10B,25A
def _eval_on_window(model, *, pos, dir_deg_val, spd, dx=0.30, dy=0.30):
    x0, y0 = pos
    left, right, back, ahead = 10.0, 10.0, 10.0, 25.0
    xmin, xmax = x0 - left, x0 + right
    ymin, ymax = y0 - back, y0 + ahead
    gx = np.arange(xmin, xmax + 1e-9, dx)
    gy = np.arange(ymin, ymax + 1e-9, dy)
    X, Y = np.meshgrid(gx, gy)

    class _TmpGrid:
        def __init__(self, X, Y): self.X, self.Y = X, Y
    old_grid = getattr(model, "grid", None)
    model.grid = _TmpGrid(X, Y)
    try:
        pos_off = model.compute_offset(pos, dir_deg_val, spd)
        Z = model.base_distribution(
            pos_xy=pos, pos_off_xy=pos_off,
            direction_deg=dir_deg_val, speed=spd, dist_from_ball=10.0
        )
    finally:
        if old_grid is not None:
            model.grid = old_grid
    return Z, (xmin, xmax, ymin, ymax)

# Placeholder if model isn’t available
def _fallback_Z(pos, dir_deg_val, spd, dx=0.30, dy=0.30):
    x0, y0 = pos
    left, right, back, ahead = 10.0, 10.0, 10.0, 25.0
    xmin, xmax = x0 - left, x0 + right
    ymin, ymax = y0 - back, y0 + ahead
    gx = np.arange(xmin, xmax + 1e-9, dx)
    gy = np.arange(ymin, ymax + 1e-9, dy)
    X, Y = np.meshgrid(gx, gy)

    th = np.deg2rad(90 - dir_deg_val)
    ct, st = np.cos(th), np.sin(th)
    s1 = 1.5 + 0.25 * spd
    s2 = 1.0
    a = (ct**2)/(2*s1**2) + (st**2)/(2*s2**2)
    b = (-2*ct*st)/(2*s1**2) + (2*ct*st)/(2*s2**2)
    c = (st**2)/(2*s1**2) + (ct**2)/(2*s2**2)
    Z = np.exp(-(a*(X-x0)**2 + b*(X-x0)*(Y-y0) + c*(Y-y0)**2))
    return Z, (xmin, xmax, ymin, ymax)

# ---------- Render ----------
def _render(*args):
    with out:
        clear_output(wait=True)
        pos = (float(x.value), float(y.value))
        spd = float(speed.value)
        ddeg = float(dir_deg.value)

        if PlayerInfluenceModel is not None:
            model = PlayerInfluenceModel(**_current_model_kwargs())
            Z, extent = _eval_on_window(model, pos=pos, dir_deg_val=ddeg, spd=spd, dx=0.30, dy=0.30)
        else:
            Z, extent = _fallback_Z(pos, ddeg, spd, dx=0.30, dy=0.30)

        fig, ax = plt.subplots(figsize=(9, 7), dpi=140)
        _draw_field_window(ax, extent)
        ax.contourf(Z, levels=int(levels.value), cmap=cmap.value, alpha=float(fill_alpha.value),
                    antialiased=True, origin="lower", extent=extent, zorder=1)
        ax.contour(Z, levels=int(levels.value), cmap=cmap.value, alpha=float(line_alpha.value),
                   linewidths=float(line_lw.value), origin="lower", extent=extent, zorder=2)

        ax.scatter(*pos, s=22, c="#0B57D0", edgecolors="white", linewidths=1.0, zorder=7)
        ax.scatter(*pos, s=8,  c="white", edgecolors="none", zorder=8)
        _add_arrow(ax, pos, ddeg, length=2.2, body=0.08, head_w=0.35, head_l=0.70, color="#2F6CFF", z=9)
        if show_o.value:
            _add_arrow(ax, pos, float(o_deg.value), length=1.8, body=0.08, head_w=0.32, head_l=0.64, color="#FF8A00", z=8)

        title = f"{name.value} – Speed={spd:.2f} yd/s"
        if show_o.value:
            title += f" | o={float(o_deg.value):.2f}°"
        ax.set_title(title, fontsize=15, pad=8, color="#2B3139")
        plt.show()

# --- Print Params handler (to params_out, not plain print) ---
def _print_params(_):
    import json, textwrap, pprint
    params = _current_model_kwargs()
    with params_out:
        clear_output(wait=True)
        # Python dict view
        print("Model params (Python dict):")
        pprint.pprint(params, sort_dicts=False)
        # JSON view
        js = json.dumps(params, indent=2)
        print("\nModel params (JSON):\n", js)
        # Try to copy JSON to clipboard in Colab
        try:
            from google.colab import output as _out  # type: ignore
            _out.eval_js(f"navigator.clipboard.writeText({json.dumps(js)})")
            print("\n✅ Copied JSON to clipboard.")
        except Exception:
            print("\nℹ️ Could not access clipboard API here. Copy manually from above.")

print_btn.on_click(_print_params)

# ---------- Preset handlers ----------
def _apply_preset(_):
    cfg = PRESETS[preset_dd.value]
    pairs = [
        (gaussian_scale_factor, "gaussian_scale_factor"),
        (alpha_gamma, "alpha_gamma"),
        (beta_min, "beta_min"),
        (beta_max, "beta_max"),
        (gamma_midpoint, "gamma_midpoint"),
        (gamma_scale_factor, "gamma_scale_factor"),
        (max_forward_distance, "max_forward_distance"),
        (forward_decay_factor, "forward_decay_factor"),
        (angle_limit_min, "angle_limit_min"),
        (angle_limit_max, "angle_limit_max"),
        (angle_decay_factor, "angle_decay_factor"),
        (w_gaussian_min, "w_gaussian_min"),
        (w_gaussian_max, "w_gaussian_max"),
        (gaussian_midpoint, "gaussian_midpoint"),
        (gaussian_steepness, "gaussian_steepness"),
        (low_speed_gaussian_cutoff, "low_speed_gaussian_cutoff"),
    ]
    for ctl, key in pairs:
        if key in cfg:
            ctl.unobserve(_render, names="value")
            ctl.value = float(cfg[key])
            ctl.observe(_render, names="value")
    _render()

def _reset(_):
    preset_dd.value = "Default1"
    _apply_preset(None)
    for ctl, val in [(speed, 5.0), (dir_deg, 0.0), (name, "Adam Thielen"), (levels, 24)]:
        if isinstance(ctl, (w.FloatSlider, w.IntSlider)):
            ctl.unobserve(_render, names="value")
            ctl.value = val
            ctl.observe(_render, names="value")
        else:
            ctl.value = val
    _render()

apply_btn.on_click(_apply_preset)
reset_btn.on_click(_reset)

# ---------- Wire controls to render ----------
controls = [
    speed, dir_deg, show_o, o_deg, x, y,
    gaussian_scale_factor, low_speed_gaussian_cutoff,
    alpha_gamma, beta_min, beta_max, gamma_midpoint, gamma_scale_factor,
    max_forward_distance, forward_decay_factor,
    angle_limit_min, angle_limit_max, angle_decay_factor,
    w_gaussian_min, w_gaussian_max, gaussian_midpoint, gaussian_steepness,
    levels, cmap, fill_alpha, line_alpha, line_lw
]
for ctl in controls:
    ctl.observe(_render, names="value")

# ---------- Link Play to speed (0→11.5 by 0.5) ----------
speed_indices = np.arange(0.0, 11.5 + 1e-9, 0.5)
def _sync_speed(change):
    idx = int(play.value)
    idx = max(0, min(idx, len(speed_indices)-1))
    speed.unobserve(_render, names="value")
    speed.value = float(speed_indices[idx])
    speed.observe(_render, names="value")
    _render()

play.observe(_sync_speed, names="value")

# ---------- Lay out with AppLayout (robust in Colab) ----------
ui = AppLayout(
    left_sidebar=left_panel,
    center=out,
    right_sidebar=None,
    pane_widths=["360px", "1fr", 0],
    height="1440px"
)

# Initial draw + show UI
_render()
display(ui)


AppLayout(children=(VBox(children=(HBox(children=(Dropdown(description='Preset', options=('Default1', 'Default…